In [2]:
%%capture
import time
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input, State, ALL, ALLSMALLER, MATCH
from dash.exceptions import PreventUpdate
import pandas as pd
poverty = pd.read_csv('../../data/poverty.csv', low_memory=False)

countries = poverty[poverty['is_country']]['Country Name'].drop_duplicates().sort_values()

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    html.Br(),html.Br(),
    dbc.Row([
        dbc.Col(lg=1),
        dbc.Col([ 
            dbc.Button("Add Chart", id='button'),

            html.Div(id='output', children=[])
            
        ], lg=4)
    ]),
])

@app.callback(Output('output', 'children'),
              Input('button', 'n_clicks'),
              State('output', 'children'))
def add_new_chart(n_clicks, children):
    if not n_clicks:
        raise PreventUpdate
    new_chart = dcc.Graph(id={'type': 'chart', 'index': n_clicks}, 
                          figure=px.bar(height=300, width=500,
                                        title=f"Chart {n_clicks}"))
    
    new_dropdown = dcc.Dropdown(id={'type': 'dropdown', 'index': n_clicks},
                                options=[{'label': c, 'value': c}
                                         for c in poverty[poverty['is_country']]['Country Name'].drop_duplicates().sort_values()])
    
    children.append(html.Div([
        new_chart, new_dropdown
    ]))
    return children

@app.callback(Output({'type': 'chart', 'index': MATCH}, 'figure'), 
              Input({'type': 'dropdown', 'index': MATCH}, 'value'))
def create_population_chart(country):
    if not country:
        raise PreventUpdate
    df = poverty[poverty['Country Name']==country]
    fig = px.line(df, x='year', y='Population, total', title=f'Population of {country}')
    return fig


app.run_server(host='0.0.0.0',port=3000)

Dash app running on http://127.0.0.1:8053/


In [ ]:
! echo 'Your app is running on: '$EDUCATIVE_LIVE_VM_URL':3000'